# Set and get hyperparameters in scikit-learn

Este caderno mostra como se pode obter e definir o valor de um hiperparâmetro em
um estimador scikit-learn. Lembramos que os hiperparâmetros se referem ao
parâmetro que irá controlar o processo de aprendizagem.

Eles não devem ser confundidos com os parâmetros ajustados, resultantes do
Treinamento. Esses parâmetros ajustados são reconhecíveis no scikit-learn porque
eles são escritos com um sublinhado final `_`, por exemplo` model.coef_`.

Começaremos carregando o conjunto de dados do censo de adultos e usaremos apenas os dados numéricos
característica.

In [9]:
import pandas as pd

adult_census = pd.read_csv("adult-census.csv")
# drop the duplicated column `"education-num"` as stated in the first notebook
adult_census = adult_census.drop(columns=['ID','fnlwgt:','education-num:'])

target_name = "class"

numerical_columns = [
    'age', 'capital-gain:', 'capital-loss:',
    'hours-per-week:']

target = adult_census[target_name]
data = adult_census[numerical_columns]

adult_census.head()

,age,workclass,education:,marital-status:,occupation:,relationship:,race:,sex:,capital-gain:,capital-loss:,hours-per-week:,native-country:,class
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Nossos dados são apenas numéricos.

In [10]:
data.head()

,age,capital-gain:,capital-loss:,hours-per-week:
0,39,2174,0,40
1,50,0,0,13
2,38,0,0,40
3,53,0,0,40
4,28,0,0,40


Vamos criar um modelo preditivo simples feito de um escalonador seguido por um
classificador de regressão logística.

Conforme mencionado em notebooks anteriores, muitos modelos, incluindo os lineares,
funcionará melhor se todos os recursos tiverem uma escala semelhante. Para este propósito,
usamos um `StandardScaler`, que transforma os dados redimensionando os recursos.

In [11]:
# to display nice model diagram
from sklearn import set_config
set_config(display='diagram')
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

model = Pipeline(steps=[
    ("preprocessor", StandardScaler()),
    ("classifier", LogisticRegression())
])
model

Pipeline(steps=[('preprocessor', StandardScaler()),
                ('classifier', LogisticRegression())])

Podemos avaliar o desempenho estatístico do modelo via
validação cruzada.

In [12]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model, data, target)
scores = cv_results["test_score"]
print(f"Accuracy score via cross-validation:\n"
      f"{scores.mean():.3f} +/- {scores.std():.3f}")

Accuracy score via cross-validation:
0.801 +/- 0.004


Criamos um modelo com o valor padrão `C` que é igual a 1. Se nós
queríamos usar um parâmetro `C` diferente, poderíamos ter feito isso quando criamos
o objeto `LogisticRegression` com algo como` LogisticRegression (C = 1e-3) `.

Também podemos alterar o parâmetro de um modelo depois que ele foi criado com
o método `set_params`, que está disponível para todos os estimadores scikit-learn.
Por exemplo, podemos definir `C = 1e-3`, ajustar e avaliar o modelo:

In [13]:
model.set_params(classifier__C=1e-3)
cv_results = cross_validate(model, data, target)
scores = cv_results["test_score"]
print(f"Accuracy score via cross-validation:\n"
      f"{scores.mean():.3f} +/- {scores.std():.3f}")

Accuracy score via cross-validation:
0.784 +/- 0.002


Quando o modelo de interesse é um `Pipeline`, os nomes dos parâmetros são do
forma `<model_name> __ <parameter_name>` (observe o sublinhado duplo no
meio). Em nosso caso, `classifier` vem da definição de` Pipeline` e
`C` é o nome do parâmetro de` LogisticRegression`.

Em geral, você pode usar o método `get_params` em modelos scikit-learn para
liste todos os parâmetros com seus valores. Por exemplo, se você quiser
obter todos os nomes de parâmetros, você pode usar:

In [14]:
for parameter in model.get_params():
    print(parameter)

memory
steps
verbose
preprocessor
classifier
preprocessor__copy
preprocessor__with_mean
preprocessor__with_std
classifier__C
classifier__class_weight
classifier__dual
classifier__fit_intercept
classifier__intercept_scaling
classifier__l1_ratio
classifier__max_iter
classifier__multi_class
classifier__n_jobs
classifier__penalty
classifier__random_state
classifier__solver
classifier__tol
classifier__verbose
classifier__warm_start


`.get_params ()` retorna um `dict` cujas chaves são os nomes dos parâmetros e cujos
os valores são os valores dos parâmetros. Se você deseja obter o valor de um único
parâmetro, por exemplo `classifier__C`, você pode usar:

In [15]:
model.get_params()['classifier__C']

0.001

Podemos variar sistematicamente o valor de C para ver se há um ótimo
valor.

In [16]:
for C in [1e-3, 1e-2, 1e-1, 1, 10]:
    model.set_params(classifier__C=C)
    cv_results = cross_validate(model, data, target)
    scores = cv_results["test_score"]
    print(f"Accuracy score via cross-validation with C={C}:\n"
          f"{scores.mean():.3f} +/- {scores.std():.3f}")

Accuracy score via cross-validation with C=0.001:
0.784 +/- 0.002
Accuracy score via cross-validation with C=0.01:
0.799 +/- 0.003
Accuracy score via cross-validation with C=0.1:
0.801 +/- 0.003
Accuracy score via cross-validation with C=1:
0.801 +/- 0.004
Accuracy score via cross-validation with C=10:
0.801 +/- 0.004


Podemos ver que, enquanto C for alto o suficiente, o modelo parece ter um desempenho
Nós vamos.

O que fizemos aqui é muito manual: envolve a varredura dos valores de C
e escolher o melhor manualmente. Na próxima lição, veremos como
para fazer isso automaticamente.

<div class="admonition warning alert alert-danger">
<p class="first admonition-title" style="font-weight: bold;">Warning</p>
<p class="last">Quando avaliamos uma família de modelos em dados de teste e escolhemos o melhor
executante, não podemos confiar na precisão da previsão correspondente, e
precisamos aplicar o modelo selecionado a novos dados. Na verdade, os dados de teste
foi usado para selecionar o modelo e, portanto, não é mais independente
a partir deste modelo.</p>
</div>

Neste caderno vimos:

- como usar `get_params` e` set_params` para obter os parâmetros de um modelo
  e defini-los.

# 📝 Exercício M3.01

O objetivo é escrever uma pesquisa exaustiva para encontrar os melhores parâmetros
combinação maximizando o desempenho estatístico do modelo.

Aqui, usamos um pequeno subconjunto do conjunto de dados do Censo Adulto para fazer o código
rápido para executar. Assim que seu código funcionar no pequeno subconjunto, tente
mude `train_size` para um valor maior (por exemplo, 0,8 para 80% em vez de
20%).

In [11]:
import pandas as pd

from sklearn.model_selection import train_test_split

adult_census = pd.read_csv("adult-census.csv")

target_name = "class"
target = adult_census[target_name]
data = adult_census.drop(columns=[target_name,'ID','fnlwgt:','education-num:', 'native-country:'])

#data['native-country:'] = data['native-country:'].replace(['Holand-Netherlands'], 'United-States')

data_train, data_test, target_train, target_test = train_test_split(
    data, target, train_size=0.2, random_state=42)

In [12]:
data.head(2)

,age,workclass,education:,marital-status:,occupation:,relationship:,race:,sex:,capital-gain:,capital-loss:,hours-per-week:
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13


In [13]:
target.head()

0    <=50K
1    <=50K
2    <=50K
3    <=50K
4    <=50K
Name: class, dtype: object

In [14]:
# to display nice model diagram
from sklearn import set_config
set_config(display='diagram')
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OrdinalEncoder

categorical_preprocessor = OrdinalEncoder()
preprocessor = ColumnTransformer(
    [('cat-preprocessor', categorical_preprocessor,
      selector(dtype_include=object))],
    remainder='passthrough', sparse_threshold=0)

# This line is currently required to import HistGradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

model = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", HistGradientBoostingClassifier(random_state=42))])

model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                                   transformers=[('cat-preprocessor',
                                                  OrdinalEncoder(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000020CD002D580>)])),
                ('classifier',
                 HistGradientBoostingClassifier(random_state=42))])

Use o modelo definido anteriormente (chamado `modelo`) e usando dois aninhados` para`
loops, faça uma busca das melhores combinações de `learning_rate` e
Parâmetros `max_leaf_nodes`. A este respeito, você precisará treinar e testar
o modelo, definindo os parâmetros. A avaliação do modelo deve ser
realizado usando `cross_val_score`. Usaremos os seguintes parâmetros
procurar:
- `learning_rate` para os valores 0,01, 0,1, 1 e 10. Este parâmetro controla
  a capacidade de uma nova árvore de corrigir o erro da sequência anterior de
  arvores
- `max_leaf_nodes` para os valores 3, 10, 30. Este parâmetro controla o
  profundidade de cada árvore.

In [15]:
from sklearn.model_selection import cross_val_score

learning_rate = [0.01, 0.1, 1, 10]
max_leaf_nodes = [3, 10, 30]

best_score = 0
best_params = {}
for lr in learning_rate:
    for mln in max_leaf_nodes:
        print(f"Evaluating model with learning rate {lr:.3f}"
              f" and max leaf nodes {mln}... ", end="")
        model.set_params(
            classifier__learning_rate=lr,
            classifier__max_leaf_nodes=mln
        )
        scores = cross_val_score(model, data_train, target_train, cv=2)
        mean_score = scores.mean()
        print(f"score: {mean_score:.3f}")
        if mean_score > best_score:
            best_score = mean_score
            best_params = {'learning-rate': lr, 'max leaf nodes': mln}
            print(f"Found new best model with score {best_score:.3f}!")

print(f"The best accuracy obtained is {best_score:.3f}")
print(f"The best parameters found are:\n {best_params}")

Evaluating model with learning rate 0.010 and max leaf nodes 3... score: 0.800
Found new best model with score 0.800!
Evaluating model with learning rate 0.010 and max leaf nodes 10... score: 0.822
Found new best model with score 0.822!
Evaluating model with learning rate 0.010 and max leaf nodes 30... score: 0.850
Found new best model with score 0.850!
Evaluating model with learning rate 0.100 and max leaf nodes 3... score: 0.848
Evaluating model with learning rate 0.100 and max leaf nodes 10... score: 0.858
Found new best model with score 0.858!
Evaluating model with learning rate 0.100 and max leaf nodes 30... score: 0.850
Evaluating model with learning rate 1.000 and max leaf nodes 3... score: 0.855
Evaluating model with learning rate 1.000 and max leaf nodes 10... score: 0.822
Evaluating model with learning rate 1.000 and max leaf nodes 30... score: 0.831
Evaluating model with learning rate 10.000 and max leaf nodes 3... score: 0.281
Evaluating model with learning rate 10.000 and 